# Imports

In [185]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Loading the data

In [186]:
df = pd.read_csv("../../data/stats/stats_processed.csv", sep=';')
df.head(5)

,winner_name,loser_name,event_date,method,round,time,time_format,bout_type,event_name,event_location,...,delta_height,delta_reach,delta_slpm_cs,delta_str_acc_cs,delta_sapm_cs,delta_str_def_cs,delta_td_avg_cs,delta_td_acc_cs,delta_td_def_cs,delta_sub_avg_cs
0,ILIA TOPURIA,MAX HOLLOWAY,26/10/2024,KO/TKO,3.0,1:34,5 Rnd (5-5-5-5-5),UFC Featherweight Title Bout,UFC 308: Topuria vs. Holloway,"Abu Dhabi, Abu Dhabi, United Arab Emirates",...,-10.16,0.00,-4.76,-1.0,-2.71,8.0,4.04,-16.0,16.0,3.9
1,KHAMZAT CHIMAEV,ROBERT WHITTAKER,26/10/2024,Submission,1.0,3:34,5 Rnd (5-5-5-5-5),Middleweight Bout,UFC 308: Topuria vs. Holloway,"Abu Dhabi, Abu Dhabi, United Arab Emirates",...,5.08,5.08,4.51,32.0,-3.34,6.0,4.24,48.0,-84.0,3.1
2,MAGOMED ANKALAEV,ALEKSANDAR RAKIC,26/10/2024,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Light Heavyweight Bout,UFC 308: Topuria vs. Holloway,"Abu Dhabi, Abu Dhabi, United Arab Emirates",...,-2.54,-7.62,-0.82,1.0,-0.45,13.0,0.43,8.0,-5.0,-0.2
3,LERONE MURPHY,DAN IGE,26/10/2024,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Featherweight Bout,UFC 308: Topuria vs. Holloway,"Abu Dhabi, Abu Dhabi, United Arab Emirates",...,5.08,5.08,-1.09,-4.0,-1.29,5.0,-0.81,6.0,-18.0,0.4
4,SHARA MAGOMEDOV,ARMEN PETROSYAN,26/10/2024,KO/TKO,2.0,4:52,3 Rnd (5-5-5),Middleweight Bout,UFC 308: Topuria vs. Holloway,"Abu Dhabi, Abu Dhabi, United Arab Emirates",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Explaining features

The features (columns) break down into 2 categories:
* **Categorical features**: Categorical features contain general fight/athlete data like: Fighters' names, nicknames, date of the event, location, number of rounds, time format, etc. Categorical features do not start with prefix `delta`, i.e. all numerical features start with prefix `delta`, the rest are categorical.
* **Numerical features**: These features contain all possible statistical details of the fight. Some of these are: The number of strikes landed, the number of significant strikes landed, the number of submissions attempted, the percentage ratio of the strikes that were landed to the head, the percentage ratio of the strikes that were landed in clinch position, etc. The numerical features, as already mentioned, can be identified by their prefix `delta`.

**Note:** Delta features can be interpreted as follows:<br>
`Delta` - is the difference between two values or metrics, often used to quantify change between different groups.<br>
So in our case `delta_feature_x` is the difference between **the winner's** `feature_x` and **the loser's** `feature_x`. <br><br>For example: <br>
There's `winner_total_strikes_landed` and `loser_total_strikes_landed`.<br>
> `delta_total_strikes_landed` = `winner_total_strikes_landed` - `loser_total_strikes_landed`. <br>

If the value of delta is *positive*, that means the **winner's** performance in that partical feature is **higher**, if the value is *negative*, it means the winner's contribution in that feature was lower compared to loser's, if it *equals 0*, it means the contribution from the winner and loser was **equal**.

* `winner_name`          - The name of the winner of the fight: ('ILIA TOPURIA', 'KHAMZAT CHIMAEV', etc.)
* `loser_name`           - The name of the loser of the fight: ('LERONE MURPHY', 'GEOFF NEAL', etc.)
* `event_date`           - The date of when event took place: ('26/10/2024', '19/10/2024', etc.)
* `winner_nickname`      - The nickname of the winner of the fight: ('El Matador', 'Borz', etc.)
* `loser_nickname`       - The nickname of the loser of the fight: ('Blessed', 'The Reaper', etc.)
* `method`               - The method of victory: ('KO/TKO', 'Submission', Decision - Unanimous', 'Decision - Split', etc.)
* `round`                - The round in which the victory was gained: ('3', '5', '1', etc.)
* `time`                 - The time of the round in which the victory was gained: ('1:34', '3:34', etc.)
* `time_format`          - Time format of the fight and how many rounds: ('5 Rnd (5-5-5-5-5)', '3 Rnd (5-5-5)', etc.)
* `referee`              - The name of the referee: ('Marc Goddard', 'Jason Herzog', etc.)
* `details`              - The details of the victory: ('Punch to Head At Distance', 'Rear Naked Choke', scorecards, etc.)
* `bout_type`            - Bout type: ('UFC Featherweight Title Bout', 'Middleweight Bout', 'Light Heavyweight Bout', etc.)
* `bonus`                - The bonus that the fight got: ('Performance of the night', 'knockout of the night', etc.)
* `event_name`           - The name of the event: ('UFC 308: Topuria vs. Holloway', 'UFC Fight Night: Hernandez vs. Pereira')
* `event_location`       - The location of the event: ('Las Vegas, Nevada, USA', 'Paris, Ile-de-France, France', etc)
* `winner_stance`        - Winner's fighting stance: ('Orthodox', 'Southpaw', etc.)
* `loser_stance`         - Loser's fighting stance: ('Switch', 'Southpaw', etc.)
* `winner`               - The fighter of which corner won: ('Red', 'Blue', etc.)
* `delta_KD`             - The delta difference in landed Knockdowns by strikes: (1,  0, -1)
* `delta_sig_str_pct`    - The delta difference in landed significant strikes in %: (14, -25,  3,)
* `delta_total_str_pct`  - The delta difference in landed total strikes in %: (significant and insignificant) : (14, -25,  3,)
* `delta_TD_pct`         - The delta difference in landed takedowns in %: (14, -25,  3,)
* `delta_sub_att`        - The delta difference in submission attempts: 
* `delta_rev`            - The delta difference in wrestling reversals (where a fighter reverses a disadvantageous position into advantageous). 
* `delta_ctrl`           - The delta difference in the fighter's control of the fight, in seconds
* `delta_sig_str_head_acc_pct` - The delta difference in accuracy (fraction of landed out of the total thrown) of landing significant strikes to the head in %
* `delta_sig_str_body_acc_pct` - The delta difference in accuracy (fraction of landed out of the total thrown) of landing significant strikes to the body in %
* `delta_sig_str_leg_acc_pct`  - The delta difference in accuracy (fraction of landed out of the total thrown) of landing significant strikes to the leg in %
* `delta_sig_str_distance_acc_pct` - The delta difference in accuracy (fraction of landed out of the total thrown) of landing significant strikes from the distance in % 
* `delta_sig_str_clinch_acc_pct` - The delta difference in accuracy (fraction of landed out of the total thrown) of landing significant strikes from the clinch in % 
* `delta_sig_str_ground_acc_pct` - The delta difference in accuracy (fraction of landed out of the total thrown) of landing significant strikes from the ground in % 
* `delta_sig_str_head_tar_pct` - The delta difference in contribution (fraction of strikes that were landed on the specified target out of the total 100% pool of 3 possible targets: head, body, legs) of significant strikes to the head in %
* `delta_sig_str_body_tar_pct` - The delta difference in contribution (fraction of strikes that were landed on the specified target out of the total 100% pool of 3 possible targets: head, body, legs) of significant strikes to the body in %
* `delta_sig_str_leg_tar_pct` - The delta difference in contribution (fraction of strikes that were landed on the specified target out of the total 100% pool of 3 possible targets: head, body, legs) of significant strikes to the leg in %
* `delta_sig_str_distance_pos_pct` - The delta difference in contribution (fraction of strikes that were landed from the specified position out of the total 100% pool of 3 possible positions: distance, clinch, ground) of significant strikes from the distance in %
* `delta_sig_str_clinch_pos_pct` - The delta difference in contribution (fraction of strikes that were landed from the specified position out of the total 100% pool of 3 possible positions: distance, clinch, ground) of significant strikes from clinch in %
* `delta_sig_str_ground_pos_pct` - The delta difference in contribution (fraction of strikes that were landed from the specified position out of the total 100% pool of 3 possible positions: distance, clinch, ground) of significant strikes from the ground in %
* `delta_height` - The delta difference in fighters' height
* `delta_reach` - The delta difference in fighters' arms reach
* `delta_slpm_cs` - The delta difference in fighters' total career statistic (computed from all fights) Significant Strikes Landed per Minute
* `delta_str_acc_cs` - The delta difference in fighters' total career statistic (computed from all fights) Significant Striking Accuracy
* `delta_sapm_cs` - The delta difference in fighters' total career statistic (computed from all fights) Significant Strikes Absorbed per Minute
* `delta_str_def_cs` - The delta difference in fighters' total career statistic (computed from all fights) Significant Strike Defence (the % of opponents' strikes that did not land)
* `delta_td_avg_cs` - The delta difference in fighters' total career statistic (computed from all fights) Average Takedowns Landed per 15 minutes
* `delta_td_acc_cs` - The delta difference in fighters' total career statistic (computed from all fights) Takedown Accuracy
* `delta_td_def_cs` - The delta difference in fighters' total career statistic (computed from all fights) Takedown Defense (the % of opponents' TD attempts that did not land)
* `delta_sub_avg_cs` - The delta difference in fighters' total career statistic (computed from all fights) Average Submissions Attempted per 15 minutes
* `delta_striking_dominance` - The delta difference in fighters' striking dominance (number of knockdowns + number of significant strikes landed + number of total strikes landed
* `delta_wrestling_dominance` - The delta difference in fighters' wrestling dominance (number of takedowns landed + number of submission attempts + number of reversals)

# Table of contents
- [Establishing Questions](#Establishing-questions)
- [Data Preprocessing](#Data-preprocessing)
- [Data Cleaning](#Data-Cleaning)
  - [NaN values](#NaN-values)
  - [Duplicates](#Duplicates)
- [Feature Enginering](#Feature-Engineering)
    - [Standardizing Features](#Standardizing-Features)
    - [Dropping Redundant Features](#Dropping-Redundant-Features)
    - [Converting Dtypes](#Converting-Dtypes)
    - [Engineering New Features](#Engineering-New-Features)
- [EDA](#EDA)
    - [1. What are the most popular locations where UFC hosts its events?](#1.-What-are-the-most-popular-locations-where-UFC-hosts-its-events?)
    - [2. What is the historical average amount of UFC fights hosted annually?](#2.-What-is-the-historical-average-amount-of-UFC-fights-hosted-annually?)
    - [3. What are the top fight outcomes?](#3.-What-are-the-top-fight-outcomes?)
    - [4. Does being in a particular corner contribute to a victory?](#4.-Does-being-in-a-particular-corner-contribute-to-a-victory?)
    - [5. What are the main fight tactics contributing to a victory?](#5.-What-are-the-main-fight-tactics-contributing-to-a-victory?)
- [Answering questions](#Answering-questions)
    - [Questions](#Questions)
    - [Answers](#Answers)

# Establishing questions

### Here are a few questions I want to be answered by the end of this notebook:
1. What are the most popular locations where UFC hosts its events?
2. What is the historical average amount of UFC events hosted annually?
3. What are the top fight outcomes?
4. Does being in a particular corner contribute to a victory?
5. Do any specific physical characteristics (like height, reach, etc.) contribute to a victory?
6. What are the main fight tactics contributing to a victory?
7. Based on the findings, can we use the information to bet on a fighter?

# EDA

## 1. What are the most popular locations where UFC hosts its events?

### Top countries by # of UFC events

Preparing the data:

In [187]:
# Dropping duplicates to keep only one event per date
events = df[['event_name', 'event_location']].drop_duplicates()

# Getting the location counts
counts = events['event_location'].value_counts()

# Make 'event_location' a column and rename the 'count' column to 'num_of_events'
top_locations = counts.reset_index().rename(columns={'count': 'num_of_events'})
top_locations['country'] = top_locations['event_location'].apply(lambda x: x.split(',')[-1].strip())
top_locations.head()

,event_location,num_of_events,country
0,"Las Vegas, Nevada, USA",245,USA
1,"Abu Dhabi, Abu Dhabi, United Arab Emirates",18,United Arab Emirates
2,"London, England, United Kingdom",15,United Kingdom
3,"Anaheim, California, USA",10,USA
4,"Atlantic City, New Jersey, USA",10,USA


In [188]:
top_countries = top_locations.copy()
# Sum by countries
top_countries = top_countries.groupby('country').sum().sort_values(by='num_of_events', ascending=False).reset_index()
# Drop column to keep it clean
top_countries.drop(columns=["event_location"], inplace=True)
top_countries.head()

,country,num_of_events
0,USA,502
1,Brazil,40
2,Canada,33
3,United Kingdom,29
4,United Arab Emirates,20


Add ISO3 country codes for plotting in choropleth:

In [189]:
import country_converter as coco
cc = coco.CountryConverter()

# Convert countries to ISO alpha-3 format
top_countries['iso_alpha'] = cc.pandas_convert(series=top_countries['country'], to='ISO3')

# Change column positions
top_countries = top_countries.iloc[:, [0, 2, 1]]
top_countries.head()

,country,iso_alpha,num_of_events
0,USA,USA,502
1,Brazil,BRA,40
2,Canada,CAN,33
3,United Kingdom,GBR,29
4,United Arab Emirates,ARE,20


Let's add log scaled version of the counts, as the range between the top1 and top 2 is huge and the choropleth gradient color is going to eat away all of the empty 40 - 500 range:

In [190]:
top_countries['log_num_of_events'] = np.log(top_countries['num_of_events'])
top_countries.head()

,country,iso_alpha,num_of_events,log_num_of_events
0,USA,USA,502,6.218600
1,Brazil,BRA,40,3.688879
2,Canada,CAN,33,3.496508
3,United Kingdom,GBR,29,3.367296
4,United Arab Emirates,ARE,20,2.995732


Plotting:

### Top countries by the number of UFC events on their soil (log-scaled)

We will add the log-scaled version of the `num_of_events` feature, as the deviation from the max value *(USA, **501**)* to the rest of the data *(Others, **1-40**)* is extremely large which would lead to choropleth stretching the color scale too much and that would make all of the other (non-USA) countries effectively the same color.

In [191]:
import plotly.express as px
import numpy as np


fig = px.choropleth(top_countries, # Plot data
                    locations='iso_alpha', # Use ISO codes as location identifiers
                    color='log_num_of_events', # Coloring based on log scaled num_of_events
                    hover_name='country', # On hover display country
                    title ="Countries by # of UFC events (log scale):",
                    color_continuous_scale=px.colors.sequential.Plasma)


# Make bigger and clearer
fig.update_layout(
    title={"x": 0.5, "font_size": 20},
    height=500, width=1000,
    geo=dict(showcountries=True),
    margin=dict(l=30, r=30, t=80, b=30)
)
fig.show()

#### Non-log version of the top 20 countries

In [192]:
fig = px.bar(top_countries[:20], x="num_of_events", y="country", color="country")


fig.update_layout(title={"text": "Top 20 countries by # of UFC Events", "x": 0.5, "font_size": 20},
                  font={"size": 14}, 
                  xaxis=dict(
                      tickmode='array', tickvals=[20, 50, 100, 200, 300, 400, 500],
                      title = "Number of events", title_standoff=25
                  ),
                  yaxis=dict(
                      title="Country", title_standoff=40
                  ),
                  
                  showlegend=False, height=500)

fig.show()

The homeland to the UFC - The USA is the clear dominant here, followed by: Brazil, Canada, the UK, the UAE, Australia, Japan, and others.

The US is obviously the first because it is, again, the homeland, also I think because of the fact that regulation concerns were cleared up over the years and overall the UFC is given a thumbs up for the events as they don't have any additional regulatory requirements to comply with as they would have to do in hosting events internationally.

That said, by looking at other countries:<br>
* Brazil is ranked at #2. And what contributes to that is the cultural roots of Brazilian jiu-jitsu, as well as the BJJ grand-master co-founder, the second top country that is the hometown to the most UFC champions, and massive fan appetite like UFC 134 in Rio selling out tickets in 74 minutes and drawing ~35 million TV viewers, which accounts to about x10 the network's average.
* Canada sits at #3. Canada is the most profitable international market for UFC, in addition to that: massive MMA fan base, popularity of one of the best UFC fighters in history, who is a canadian, George St-Pierre, and strong Canadian regulatory support after legalization in major provinces make it the #2.
* UK at #4 is the premiere MMA hub in Europe (London, Manchester). Loyal fanbase, home to stars like Bisping, Aspinall; Prime timezone for capturing European/US crossover audiences and provides fertile ground for local talent development, e.g. Cage Warriors (the UK-based MMA promotion) to UFC pipeline, e.g. Conor McGregor.
* #5, UAE. UAE isn't known for huge MMA fanbases, local stars or notably strong PPV numbers, but it has its own reasons. Massive government funding and infrastructure support, e.g. they helped UFC build "Fight Island" on Yas Island during COVID. As well as, showcases MMA as a global sport to regional audiences.
* Australia at #6. Australian fans are known to be loud, loyal, and spend-heavy. Local champions like Alexander Volkanovski, Israel Adesanya, Robert Whittaker keep demand high.
* Japan, #7. Japan was once the global center of MMA - Pride FC, Shooto, Pancrase. So UFC tried to capture that market and attempted expansion in early 2000s, then again with UFC Japan events in late 2010s. It is known for deep martial arts culture and knowlegable fans.

But let's take a look at the ratio that it has with respect to the total pool of events in all of the countries:

In [193]:
# Calculate what pct of UFC events take place in the US
usa_pct = (top_countries["num_of_events"] / sum(top_countries["num_of_events"]) * 100).iloc[0]

# Prep the data
pct = pd.DataFrame({"country": ["USA", "Others"], "pct_of_events": [usa_pct, 100 - usa_pct]})
pct

,country,pct_of_events
0,USA,70.803949
1,Others,29.196051


Plot:

In [194]:
fig = px.pie(pct, values="pct_of_events", names="country",
             title="Percentage of # of UFC events taking place in the US vs others", height=500)

fig.update_layout(title={"x": 0.5, "font_size": 20},
                  font={"size": 14})

fig.update_traces(textfont_size=14, textfont={'color': 'white'})

fig.show()

70.8% is the USA only! Alright, but let's take a look at the states:

### Top US states by # of UFC events

#### Preparing the data

Let's prep the data:

In [195]:
# Get a separate df for the top US states
top_states = top_locations[top_locations['country'] == 'USA'].copy()
# Extract the states from 'event_location' column
top_states['state'] = top_states['event_location'].apply(lambda x: x.split(',')[-2].strip())
# Count the number of events per state and sort
top_states = top_states.groupby('state').sum('num_of_events').sort_values(by='num_of_events', ascending=False).reset_index()

# A dictionary that maps full state names to their abbreviations
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "DC": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "Virgin Islands, U.S.": "VI",
}

# Mapping to abbreviations
top_states['state_abv'] = top_states['state'].apply(lambda x: us_state_to_abbrev[x])
top_states.head()

,state,num_of_events,state_abv
0,Nevada,245,NV
1,California,33,CA
2,Texas,25,TX
3,Florida,22,FL
4,New Jersey,22,NJ


### Plotting

In [196]:
top_states['log_num_of_events'] = np.log(top_states["num_of_events"])

In [197]:
import plotly.express as px
import numpy as np


fig = px.choropleth(top_states,
                    locations="state_abv",
                    locationmode="USA-states",
                    color='num_of_events',
                    scope="usa",
                    title="The US states by # of UFC events",
                    color_continuous_scale=px.colors.sequential.Plasma)

fig.add_scattergeo(
    locations=top_states["state_abv"],
    locationmode="USA-states",
    text=top_states["state_abv"],
    mode="text",
    textfont=dict(
        size=12,
        color='red'
    ),
)

# Make bigger and clearer
fig.update_layout(
    title={'x': 0.5, "font_size": 20},
    height=500, width=1000,
    geo=dict(showcountries=True),
    margin=dict(l=30, r=30, t=80, b=30)
)
fig.show()

In [198]:
fig = px.bar(top_states[:20], x="num_of_events", y="state", color="state")


fig.update_layout(title={"text": "Top 20 countries by # of UFC Events", "x": 0.5, "font_size": 20},
                  font={"size": 14}, 
                  xaxis=dict(
                      tickmode='array', tickvals=[20, 50, 100, 200, 300, 400, 500],
                      title = "Number of events", title_standoff=25
                  ),
                  yaxis=dict(
                      title="Country", title_standoff=40
                  ),
                  
                  showlegend=False, height=500)

fig.show()

Let's take a look at a few:
* Nevada is #1 as Las Vegas is the fight capital of the world, it is considered the home of boxing since 1960s (Muhammad Ali, Mike Tyson, Mayweather), the biggest fights are usually expected to be in Vegas. So UFC benefits from Las Vegas's established reputation as the world's top fight city. Some other reasons are that UFC is headquartered in Las Vegas, Nevada's State Athletic Commission being fighter-friendly and high tourism aspect.
* California at #2 because of the largest population in the U.S. has huge fanbase and ticket demand, also CA has been longtime MMA-friendly since it was the first state to regulate MMA (1990s) also LA events are very good for media eyeballs.
* Texas at #3 because of the low-friction regulation, drug testing and licencing is usually easier in Texas. Additionally, Texas offers stadium-sized venues like Toyota Center and passionate fanbase.
* Florida at #4 also because of low regulatory risk, and one a place for capturing Latin/Caribbean fanbase.
* New York at #6 is interesting, even though the strictest regulation, UFC literally lobbied 10+ years to legalize MMA, which finally passed in 2016. NY is prestige & media, MSG is known as "World's most famous arena", NY cards are often loaded PPVs.

## 2. What is the historical average amount of UFC fights hosted annually?

Preparing the data for plotting. Dropping duplicates, saving only the annual data, and then sorting it:

In [199]:
# Drop duplicates
events_annually = df[['event_name', 'event_date']].drop_duplicates()
# Extracting annual data
events_annually['event_year'] = pd.to_datetime(events_annually['event_date'], dayfirst=True).dt.year

# Sorting 
events_annually_counts = events_annually['event_year'].value_counts(sort=False)
# Making it look more presentable
events_annually_counts = events_annually_counts.reset_index().rename(columns={'count': 'num_of_events'})

# Taking a look
events_annually_counts.head()

,event_year,num_of_events
0,2024,36
1,2023,43
2,2022,42
3,2021,43
4,2020,41


Let's plot this:

In [200]:
fig = px.line(events_annually_counts, x='event_year', y='num_of_events', markers=True)

# Cosmetics config
fig.update_layout(title={"text": "Number of UFC events per year", "x": 0.5, "font_size": 20},
                  font={"size": 14}, 
                  xaxis=dict(
                      tick0=1994, # this specifies the x axis tick to start from
                      dtick=3, # this specifies the step of x axis ticks
                      title = "Year", title_standoff=25
                  ),
                  yaxis=dict(
                      title="Number of events", title_standoff=40
                  ),
                  height=500)

# Increase the width of the line
fig.update_traces(patch={"line": {"width": 2.5}})
fig.show()

We can observe the following:
* The chart starts out with initial steady growth throughout 1995-2005, fluctuating of 2-3 events on average;
* Then, the 2005-2010 era, is where the UFC signed a deal with Spike TV to broadcast The Ultimate Fighter show, at the same time UFC starts being taken as a legitimate sports property, shifting perception from the "human cockfighting" to "real sport" in controlled, regulated environment, UFC starts expanding.
* In 2011-2014 UFC enters Brazil, Asia, Europe - showcased by a jump from 27 => 46 events.
* In 2015-2019 there's steady number of annual events, ~40 on average, UFC signs deal with ESPN, becomes more structured, stable, profitable.
* 2020-2021 is interesting, it's the COVID era, but UFC did well, due to having build UFC Apex (UFC's own custom facility that is adapted to hosting events, as well as a training facility) a year before in 2019 as well as Abu Dhabi's built fight island. This allows them to continue hosting events behind closed doors (no other leage could do that).
* 2021-2024 shows how fast UFC normalized their events, where Florida was the first state to allow them continue hosting events
* 2024 is still going so we're not going to draw any conclusions here just yet.

## 3. What are the top fight outcomes?

Getting the data ready:

In [201]:
# Get the counts per every victory method
top_fight_outcomes = df['method'].value_counts()

# Extracting data
outcomes = top_fight_outcomes.index
counts = top_fight_outcomes.values

# Taking a peak
top_fight_outcomes

method
Decision - Unanimous       2793
KO/TKO                     2484
Submission                 1550
Decision - Split            750
TKO - Doctor's Stoppage      94
Decision - Majority          60
DQ                           23
Name: count, dtype: int64

Plotting it:

In [202]:
fig = px.bar(x=counts, y=outcomes, color=outcomes)

# Cosmetics config
fig.update_layout(title={"text": "Top fight result methods", "x": 0.5, "font_size": 20},
                  font={"size": 14}, 
                  xaxis=dict(
                      title = "Count", title_standoff=25
                  ),
                  yaxis=dict(
                      title="Method", title_standoff=40
                  ),
                  showlegend=False, height=500)
fig.show()

* The top 1 result is a win by Decision - Unanimous. The reason for that is that: 1 - it is much harder to KO/TKO or submit an opponent as it usually indicates a significant difference in expertese / level of fighters, and 2 - is probably the fact that a lot of the fighters like to "play safe", avoid taking big risks and just win by points.
* The top 2 is KO/TKO, followed by submission at top 3, which is just a grappler's way of KO/TKOing someone. KO/TKO is more common due to two reasons: 1 - the fights, as well as each round, start in stand-up (from striking position) and not from wrestling position, meaning on average, fighters spend more time striking than grappling, 2 - There is more strikers than wrestlers in the UFC roster, and potentially 3 - Fighters might, sometimes, avoid wrestling as the fans do not like it and say that it is "boring" and prefer striking instead.
* At top 4 - Decision - Split, is when only 2 judges out of 3 gave the victory to the winner. Usually happens relatively more rare, as the judges are, usually, on the same page when it comes to a winner.
* At top 5 - TKO Doctor's Stoppage - the result, when a fighter is deemed unfit by a doctor or referee to continue to fight due to an injury, even less likely, because, usually, small cuts or swellings that could naturally happen during the fight are not deemed to unfit to continue, and it is marginally much rarer that a fighter sustained a significant injury and is not knocked out or submitted.
* At top 6 - Decision - Majority, almost the same as Decision - Split, but instead of the 2 out of 3 judges giving victory to the winner and the 3rd giving victory to the loser, in Majority Decision the third judge scores the fight as draw. The majority decisions are less common because it is harder statistically, one fighter would have to be scored two 10-8 rounds and then the other fighter one 10-8 round in a 3-rd fight, which is rarer, or a round being scored 10-10 which is even less likely as judges are discouraged to do that, unless it is impossible to assign the winner of the round, even no slightest difference can be noted, or a combination of the factors above + point deductions, which happen in fewer than 2% of the fights.
* At top 7 - DQ, DQ is disqualification, it is the rarest, as the fighters are usually very careful when it comes to the rules, and disqualification effectively means loss, so they'd rather try and win even if they're losing than outright lose.

## 4. Does being in a particular corner contribute to a victory?

Data:

In [203]:
# Sorting
winners_by_corner = df['winner'].value_counts()

# Converting into pct
winners_by_corner = winners_by_corner / sum(winners_by_corner) * 100

Let's plot:

In [204]:
winners_by_corner = pd.DataFrame(winners_by_corner).reset_index()
winners_by_corner

,winner,count
0,red,65.0374
1,blue,34.9626


In [205]:
fig = px.pie(winners_by_corner, values='count', names='winner',
             title="Total % of winners in the corresponding corner", height=500,
            color = 'winner', color_discrete_map={'red':'EF553B','blue':'636EFA'})

fig.update_layout(title={"x": 0.5, "font_size": 20},
                  font={"size": 14}, )

fig.update_traces(textfont_size=14, textfont={'color': 'white'})

fig.show()

* The red corner is the dominant since it is given to champions, and, higher-ranked fighters.
* Where blue is assigned to lower-ranked fighters and title challengers.

## 5. What are the main fighting aspects contributing to a victory?

### Let's examine a few features between a winner and a loser:

#### Height

**Univariate analysis**

For the starters, let's examine some anatomy-based features like height and reach. <br>
Let's start with `height`:

In [206]:
df['delta_height'].describe()

count    6813.000000
mean        0.298999
std         6.522503
min       -33.020000
25%        -5.080000
50%         0.000000
75%         5.080000
max        33.020000
Name: delta_height, dtype: float64

Looks like we have some outliers here. The mean is 0, std is 11, but min and max are -193, 193, which looks like they are the outliers, these are essentially the cases, where the height of one of the fighters is not recorded, and so is filled with 0. Effectively taking delta values by subtracting 0 or from 0, resulting in the height of a single fighter, let's drop these:

But before dropping, we will write a function as I suspect there's a bunch similar not-filled entries in other columns as well, and we cannot simply drop them all at once, as there's still a lot of useful information in those rows in other columns:

In [207]:
def drop_outliers(series, outlier_threshold):
    """Returns series that dropped rows, 
    where values are above the threshold"""
    
    mask = np.abs(series) > outlier_threshold
    idxs = mask[mask].index

    return series.drop(index=idxs)

In [208]:
dtheights = drop_outliers(df['delta_height'], 100)
dtheights.describe()

count    6813.000000
mean        0.298999
std         6.522503
min       -33.020000
25%        -5.080000
50%         0.000000
75%         5.080000
max        33.020000
Name: delta_height, dtype: float64

Looks much better, let's plot:

In [209]:
fig = px.box(dtheights, x='delta_height', height=500, title="Differences in heights between winner and loser")

fig.update_layout(title={"x": 0.5, "font_size": 20},
                  font={"size": 14}, )
fig.show()

#### Reach

In [210]:
df["delta_reach"].describe()

count    5705.000000
mean        0.573893
std         8.241648
min       -30.480000
25%        -5.080000
50%         0.000000
75%         5.080000
max        33.020000
Name: delta_reach, dtype: float64

Looks like we have some outliers here as well. The mean is 17, std is 63, but min and max are -205, 213, it is anatomically, impossible to have a difference between fighters' reach of even 100. So these are just the rows, where one of the fighters' reach wasn't filled. So let's drop these:

In [211]:
dtreach = drop_outliers(df["delta_reach"], 100)
dtreach.describe()

count    5705.000000
mean        0.573893
std         8.241648
min       -30.480000
25%        -5.080000
50%         0.000000
75%         5.080000
max        33.020000
Name: delta_reach, dtype: float64

In [212]:
fig = px.box(dtreach, x='delta_reach', height=500, title="Differences in reach between winner and loser")

fig.update_layout(title={"x": 0.5, "font_size": 20},
                  font={"size": 14}, )
fig.show()

#### Fighting stance

Now, let's take a look at whether fighting stance matters.

All possible stances:

In [213]:
set(df.winner_stance.unique().tolist() + df.loser_stance.unique().tolist())

{'Open Stance', 'Orthodox', 'Sideways', 'Southpaw', 'Switch', nan}

Let's drop the 0's:

In [214]:
cols = [['winner_stance', 'loser_stance']]
idxs = df[(df['winner_stance'] == "0") | (df['loser_stance'] == "0")].index

stances_df = df[['winner_stance', 'loser_stance']].drop(index=idxs).dropna()

In [215]:
stances_df['winner_stance'].value_counts()

winner_stance
Orthodox       4937
Southpaw       1416
Switch          315
Open Stance      13
Sideways          2
Name: count, dtype: int64

In [217]:
stances_df['loser_stance'].value_counts()

loser_stance
Orthodox       5101
Southpaw       1287
Switch          280
Open Stance      11
Sideways          4
Name: count, dtype: int64

Let's calculate win rate for each stance:

In [218]:
# Stances total
stances_total = stances_df['winner_stance'].value_counts().add(stances_df['loser_stance'].value_counts(), fill_value=0)

# Stances win rate
stances_wr = (stances_df['winner_stance'].value_counts().div(stances_total, fill_value=0) * 100).sort_values()
stances_wr

winner_stance
Sideways       33.333333
Orthodox       49.183104
Southpaw       52.386238
Switch         52.941176
Open Stance    54.166667
Name: count, dtype: float64

Good.

Let's examine the different stance matchups:

In [219]:
stances_df.value_counts()

winner_stance  loser_stance
Orthodox       Orthodox        3814
Southpaw       Orthodox        1056
Orthodox       Southpaw         917
Southpaw       Southpaw         295
Switch         Orthodox         220
Orthodox       Switch           196
Switch         Southpaw          72
Southpaw       Switch            61
Switch         Switch            23
Open Stance    Orthodox          10
Orthodox       Open Stance        7
Southpaw       Open Stance        3
Open Stance    Southpaw           3
Orthodox       Sideways           3
Southpaw       Sideways           1
Sideways       Open Stance        1
               Orthodox           1
Name: count, dtype: int64

In [220]:
# Pick rows, where the stances of both fighters are the same
samestance_idxs = stances_df[stances_df.winner_stance == stances_df.loser_stance].index

# Drop same stance matchups
stances_df.drop(index=samestance_idxs, inplace=True)

In [221]:
matchup_cnts = stances_df.value_counts().reset_index()
matchup_cnts

,winner_stance,loser_stance,count
0,Southpaw,Orthodox,1056
1,Orthodox,Southpaw,917
2,Switch,Orthodox,220
3,Orthodox,Switch,196
4,Switch,Southpaw,72
5,Southpaw,Switch,61
6,Open Stance,Orthodox,10
7,Orthodox,Open Stance,7
8,Open Stance,Southpaw,3
9,Orthodox,Sideways,3


Normalize winner/loser_stance Southpaw:Orthodox and loser/winner_stance Orthodox:Southpaw into a single group Southpaw:Orthodox to get the total counts needed for ratio calculations:

In [223]:
stances_df['stance_pairs'] = stances_df.apply(lambda row: tuple(sorted([row['winner_stance'], row['loser_stance']])), axis=1)
stance_pairs_cnts = dict(stances_df['stance_pairs'].value_counts())
stance_pairs_cnts

{('Orthodox', 'Southpaw'): 1973,
 ('Orthodox', 'Switch'): 416,
 ('Southpaw', 'Switch'): 133,
 ('Open Stance', 'Orthodox'): 17,
 ('Open Stance', 'Southpaw'): 6,
 ('Orthodox', 'Sideways'): 4,
 ('Sideways', 'Southpaw'): 1,
 ('Open Stance', 'Sideways'): 1}

In [ ]:
stances_df['stance_pairs_cnts'] = stances_df['stance_pairs'].apply(lambda val: stance_pairs_cnts[val])
stances_df.head()

In [ ]:
matchup_cnt = pd.DataFrame(stances_df[['winner_stance', 'loser_stance']].value_counts()).reset_index()
matchup_cnt

In [ ]:
stances_df = stances_df.merge(matchup_cnt, how='left', on=['winner_stance', 'loser_stance']).rename(columns={'count': 'matchup_cnt'})
stances_df

In [ ]:
stances_df.drop_duplicates(subset=['loser_stance', 'winner_stance'], inplace=True)

# Calc win rate
stances_df['stance_matchups_wr'] = (stances_df.matchup_cnt.div(stances_df.stance_pairs_cnts) * 100).round(2)

# Drop now irrelevant cols for interpretability
stances_df = stances_df.loc[:, ['winner_stance', 'loser_stance', 'stance_matchups_wr']]
stances_df

In [ ]:
stances_df

In [ ]:
stances_df['matchup_cnt'] = stances_df[['winner_stance', 'loser_stance']].value_counts()

In [771]:
name, cnt = counts.index[0], counts.values[0]

11           (Orthodox, Switch)
12                         1941
15           (Orthodox, Switch)
17                         1941
19                         1941
                 ...           
6634                       1941
6635                       1941
6641                       1941
6644                       1941
6645    (Open Stance, Orthodox)
Name: stance_pairs, Length: 2487, dtype: object

In [682]:
buf = stances_df['stance_pairs'].iloc[0]
buf

('Orthodox', 'Switch')

In [685]:
stances_df[stances_df['stance_pairs'] == buf] = 1

11                            1
12         (Orthodox, Southpaw)
15                            1
17         (Orthodox, Southpaw)
19         (Orthodox, Southpaw)
                 ...           
6634       (Orthodox, Southpaw)
6635       (Orthodox, Southpaw)
6641       (Orthodox, Southpaw)
6644       (Orthodox, Southpaw)
6645    (Open Stance, Orthodox)
Name: stance_pairs, Length: 2487, dtype: object

Let's start plotting:

In [ ]:
# Mean, std, min/max, quartiles labels
mean_and_std_labels = df['winner_striking_dominance'].describe().index[1:].array


# Function to plot a kde histogram
def examine_feature_kde(feature_name, log_transform=False):
    fig, ax = plt.subplots(figsize=(10, 5))
    plt.title(f"Winner_{feature_name} vs Loser_{feature_name}", fontdict={'fontsize':14})

    # Two histograms to compare against each other, the values are log1p transformed if the corresponding argument is passed
    sns.histplot(np.log1p(df[f'winner_{feature_name}']) if log_transform else df[f'winner_{feature_name}'], color='red', 
                kde=True, bins=120, label=f'winner_{feature_name}')
    sns.histplot(np.log1p(df[f'loser_{feature_name}']) if log_transform else df[f'loser_{feature_name}'], color='blue', 
                kde=True, bins=120, label=f'loser_{feature_name}')

    # Plot config
    ax.set_xlabel(f"{feature_name}", fontdict={'fontsize': 12})
    ax.set_ylabel("Count", fontdict={'fontsize': 12})
    plt.legend()
    plt.show()


# Function to plot descriptive stats bar chart
def examine_feature_mean_std(feature_name):
    # Winner values
    winner_feature_values = df[f'winner_{feature_name}'].describe().values[1:]
    # Loser values
    loser_feature_values = df[f'loser_{feature_name}'].describe().values[1:]

    # Plotting
    plt.figure(figsize=(10,5))
    # X ticks
    x = range(len(mean_and_std_labels))

    # Plots
    plt.bar(x, winner_feature_values, color='red', width=0.5, align='center', label=f'winner_{feature_name}')
    plt.bar(x, loser_feature_values, color='blue', width=0.5, align='edge', label=f'loser_{feature_name}')

    # Plot config
    plt.xticks(x, mean_and_std_labels)
    plt.title(f"Mean, STDs and quartiles of the winner/loser {feature_name}", fontdict={'fontsize': 14})
    plt.xlabel("Data points", fontdict={'fontsize': 12})
    plt.ylabel("Counts", fontdict={'fontsize': 12})

    # Plot a legend
    plt.legend()
    plt.show()

#### Striking Dominance

In [ ]:
examine_feature_kde('striking_dominance')

Observations:
* We can clearly see that winners have a higher striking dominance on average, with a higher peak, and also a mean being around 110, whereas losers have a mean of around 80, we will take a look at the exact numbers very shortly;
* There is a significant overlap in the 75-100 striking dominance factor range, indicating a significant striking opposition;
* We can also see a large number of losers, as well as winners, having around 0 striking dominance, most likely due to being knocked out or submitted in the first round with having thrown zero punches, also the lack of data from the old events is possible, where they did not record these data points. We will look deeper into it in just a minute;
* There is also, what looks like a capped bin at the 200 range, which is obvious, because the feature "striking dominance factor" consists of two added pct % based features, which can possibly have the maximum value of 100%, thus 100+100=200 bin. (Also the KD feature plays the role in the feature creation, which is the number of knockdowns, and by default is very low: in the 0-5 range, thus the bin is not exactly at 200, but more like 200-205).


Let's check the mean and std:

In [ ]:
examine_feature_mean_std('striking_dominance')

Observations:
* As I said, the mean difference is about 20 points;
* Standard deviation is a bit higher for losers, indicating a little higher variability for losers;
* All quartiles are bigger for the winner, indicating a clear striking contribution to the fight outcome.

Let's investigate deeper whether the big bin with striking dominance factor 0 is due to the losers being knocked out in the first round or lack of data:

In [ ]:
# Saving samples, where the striking dominance is 0
zero_winner_striking_dominance = df.loc[df['winner_striking_dominance'] == 0, ['round']].value_counts()
zero_loser_striking_dominance = df.loc[df['loser_striking_dominance'] == 0, ['round']].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].pie(zero_winner_striking_dominance, labels=[1,2], autopct="%.1f%%")
ax[1].pie(zero_loser_striking_dominance, labels=[1,2], autopct="%.1f%%")
ax[0].set_title("winner")
ax[1].set_title("loser")

plt.legend(['round 1', 'round 2'])
plt.tight_layout(pad=5)
fig.suptitle("Rounds in which the x_striking_dominance is 0", fontsize=16)
plt.show()

* As we can see, around 97% of 0 striking dominance factor comes from fights that ended in the first round;
* There is still a small percentage of 0 striking dominance factor that comes from the fights that ended in the second round. I think the reason for this is just the lack of data points that the striking_dominance features were built upon, as the older UFC fights didn't have this detailed data, so let's take a look:

Winner striking dominance that has a value of 0 and happened in the second round:

In [ ]:
df.loc[(df['winner_striking_dominance'] == 0) & (df['round'] == 2), :]

Winner striking dominance that has a value of 0 and happened in the second round:

In [ ]:
df.loc[(df['loser_striking_dominance'] == 0) & (df['round'] == 2), :]

Observations:
* So, as we assumed, 4 empty data fights are from the 1996 and 1998;
* And the 2008 fight from the empty **loser_striking_difference** is an example, where a losing fighter could not land a single shot and, eventually, got knocked out.

But let's get actually deeper into the features and see exactly what striking attributes contribute to winning.

#### KD (Knockdowns)

In [ ]:
examine_feature_kde('KD')

Observations:
* We can see a higher peak at the 0 range for losers, indicating losers not score Knockdowns as much as winners;
* We can see a higher peak at 1 range for winners, indicating winners scoring Knockdowns more;
* There is a higher peak for 2 range as well, where winners score 2 knockdowns more than losers;
* Winners also have a longer tail, indicating higher outliers compared to losers.

In [ ]:
examine_feature_mean_std('KD')

Observations:
* We can see a clear pattern. The mean, standard deviation, 3rd quartile, and max are all higher for the winners, which indicates a correlation to victory;
* Min, 1st, and 2nd quartiles are null due to half of the data having 0 for knockdowns, as they're not that common, a small amount of missing data is also possible.

#### Significant strikes percentage

In [ ]:
examine_feature_kde('sig_str_pct')

Observations:
* A higher peak for losers at 0 range, indicating that winners score more significant strikes;
* A little bit higher peak for losers at 100 range, where, likely, the winners were out wrestling their opponents or ended the fight with a  KO/TKO/Submission. We will investigate this in a minute;
* Winner's distribution is skewed higher. Indicating that winners land more significant strikes on average.

In [ ]:
examine_feature_mean_std('sig_str_pct')

Observations:
* Indeed, as we can see, every single data point except for standard deviation is higher for winners, which indicates a correlation with the result of the fight;
* Standard deviation is a bit higher for losers, around a 2-point difference, indicating that values range just a little bit more for losers in this feature.

#### Total strikes landed percentage

In [ ]:
examine_feature_kde('total_str_landed_pct')

Observations:
* Again, a higher peak for losers at 0 range, indicating that losers are more prone to land strikes;
* Also again, we can observe a little bit higher peak for losers at the 100 range, again, likely due to winners focusing more on wrestling, or where losers lost by a KO/TKO/Submission;
* Winner's distribution is skewed higher. Indicating that winners land more total strikes on average.

In [ ]:
examine_feature_mean_std('total_str_landed_pct')

Observations:
* The same pattern so far, all data points except for standard deviation are bigger for winners compared to losers, indicating a correlation between this feature and the result of the fight.
* Standard deviation is a bit higher for losers, around a 2-point difference, indicating that values range just a little bit more for losers in this feature.

#### Significant strikes to the head percentage

In [ ]:
examine_feature_kde('sig_str_head_pct')

Observations:
* The same pattern here, a much higher peak at 0 range for losers, which indicates that losers deal less significant strikes to the head;
* A bit higher peak at tick 100 for winners, indicating that winners strike 100% to the head only a bit more often than losers;
* Winner's distribution is skewed higher, indicating that winners land more significant strikes to the head on average.

In [ ]:
examine_feature_mean_std('sig_str_head_pct')

Observations:
* Again, all data points except std are winner-dominant. Indicates a correlation with the fight result.
* Standard deviation is a bit higher for losers, around a 5-point difference, indicating that values range just a little bit more for losers in this feature.

#### Significant strikes to the body percentage

In [ ]:
examine_feature_kde('sig_str_body_pct')

Observations:
* The same pattern here, a much higher peak at tick 0 for losers, which indicates that losers are more prone to land 0 significant strikes to the body;
* Now it's getting interesting, looks like the loser's distribution is skewed higher for this feature;
* Also, a much longer tail for losers, indicating that losers are more prone to land a lot of their significant strikes to the body than winners;


In [ ]:
examine_feature_mean_std('sig_str_body_pct')

Observations:
* The mean and quartiles confirm this. All of the data points, except for the first quartile are higher for losers than for winners. Looks like losers indeed do tend to land their significant strikes on the body more than winners.
* Standard deviation is at around 3 points difference, which makes sense because the losers' distribution has longer tails.

#### Significant strikes to legs percentage

In [ ]:
examine_feature_kde('sig_str_leg_pct')

Observations: 
* Very close peaks on tick 0, indicate that neither winners nor losers really focus their significant strikes on the leg, and instead, they are more likely to go for the head and the body;
* But look at this, loser's distribution here is also skewed higher, indicating that losers on average land more significant strikes to the leg;
* Also the long tail, indicates a few outliers, where the losers have a much higher peak, where all of their significant strikes went to the leg region.

In [ ]:
examine_feature_mean_std('sig_str_leg_pct')

Observations:
* Mean, std, and quartiles confirm this. All of the data points are higher for losers;
* Standard deviation is higher for losers, due to a longer tail.

#### Significant strikes from the distance percentage

In [ ]:
examine_feature_kde('sig_str_distance_pct')

Observations:
* A higher peak at tick 0 for losers, indicating that they have more samples of scoring zero significant strikes from the distance, most likely due to a first-round loss;
* But also a higher peak at tick 100, indicating that when they do score significant strikes from the distance, they mostly strike from that position, whereas winners probably mix things up, like striking from the clinch or the ground. We will see this in just a bit;
* Winners like to strike from the distance as well, especially at the 20 - 60% range, but then for the rest of their striking they likely choose to land from other positions.

In [ ]:
examine_feature_mean_std('sig_str_distance_pct')

Observations:
* Descriptive statistics confirm this. All of the data points are higher for losers, which indicates that losers like to keep it simple, and mostly strike from a distance;
* Standard deviation is almost the same, with a 1-2 point difference.

#### Significant strikes from clinch percentage

In [ ]:
examine_feature_kde('sig_str_clinch_pct')

Observations:
* We can see a higher peak at tick 0 for losers, with a difference of around 500 samples, which indicates that losers are more prone to not strike from the clinch at all, but also losing in the first round could be the reason here;
* We can observe a small difference in the 0 - 20% range, where winners land their significant strikes from the clinch just a bit more than losers;
* But then a long tail, where some outliers are present for losers scoring 100% of their significant strikes from the clinch position.

In [ ]:
examine_feature_mean_std('sig_str_clinch_pct')

Observations:
* This one is close. Still, all data points are in the losers' favor, but the difference is very small, with a 2 - 4 points difference on average;
* Standard deviation is also higher for losers with an around 3 points difference, due to the long tail.

#### Significant strikes from the ground percentage

In [ ]:
examine_feature_kde('sig_str_ground_pct')

Observations:
* Woah, a lot of samples at 0 tick here, likely due to such factors as:
    * A potentially bigger amount of strikers in the roster than wrestlers;
    * Fights starting on the feet;
    * Fighters prefer to strike than wrestle, as that's what the public likes;
* Longer tail indicates winners' choice of taking their opponent down, which is correlated to winning, and is logical as wrestlers win more than losers on average (out of the scope of this dataset).

In [ ]:
examine_feature_mean_std('sig_str_ground_pct')

Observations:
* Descriptive statistics confirm this. All of the data points are higher for winners. Winners like to wrestle more than losers.
* Standard deviation is higher for winners due to the long tail.

#### Wrestling dominance

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 5))
plt.suptitle("Winner wrestling dominance vs loser wrestling dominance", fontsize=14)

plt.subplot(2,1,1)
sns.histplot(df['winner_wrestling_dominance'], color='green', kde=True, bins=120, label='winner')
plt.xlabel("winner wrestling dominance ", fontdict={'fontsize': 12})
plt.ylabel("count", fontdict={'fontsize': 12})

plt.subplot(2,1,2)
sns.histplot(df['loser_wrestling_dominance'], color='blue', kde=True, bins=120, label='loser')
plt.xlabel("loser wrestling dominance", fontdict={'fontsize': 12})
plt.ylabel("count", fontdict={'fontsize': 12})

plt.subplots_adjust(hspace=0.4)
fig.legend()
plt.show()

Observations:
* We can see that the most of wrestling dominance factor is around 0 for both, winners, and losers. This indicates that fighters don't wrestle as much, as strike, also, since the fight starts on the feet and not on the ground;
* Even though a lot of wrestling dominance at the 0 range has both winners and fighters, winners have around 1.5 less wrestling dominance around the 0 range than losers, losers' peak is around 4000, while winners' is around 2500);
* Both distributions have long tails extending to higher values of dominance, indicating a few fighters display extreme wrestling dominance;
* Winners have slightly more density in the higher dominance range than losers, implying that the higher wrestling dominance might correlate to winning.
* Extreme outliers show that winners have slightly more extreme wrestling dominance than losers at the 100 range.

Let's check the mean, std, min/max and quartiles:

In [ ]:
examine_feature_mean_std('wrestling_dominance')

Observations:
* Winners have a higher mean, around 10 points difference, meaning that on average they have a higher wrestling dominance score;
* Winners have a higher standard deviation, meaning that the data deviates from the mean a little bit more compared to losers' std;
* The first quartile is empty, due to a lot of samples being at 0;
* The second quartile (50%): unlike winners', losers' quartile is still 0, due to a bigger number of samples at the 0 range;
* The second and third (50% and 75%) quartiles confirm the bigger density for winners.

#### Takedown percentage

In [ ]:
examine_feature_kde('TD_pct')

Observations: 
* A higher peak at tick 0, indicates that loses land at least one takedown less often than winners;
* We can also see a higher peak at 50 and 100 ticks range, where winners are scoring more takedowns.

The scale of the data is hard to see, let's get into descriptive statistics before we jump to any conclusions:

In [ ]:
examine_feature_mean_std('TD_pct')

Observations: 
* This is much more clear now. All of the data points are higher for winners, and the mean, std, and quartiles are all higher for winners, which indicates that scoring more takedowns is correlated to winning;
* Standard deviation is around 5 points higher for winners, as they have fewer samples at 0 tick and more samples at tick 100.

#### Submission attempts

In [ ]:
examine_feature_kde('sub_att')

Observations:
* A higher peak at a 0 tick, again, indicating that losers perform fewer submission attempts;
* Also higher peaks at 1, 2, 3, and 4 ticks, clearly show a tendency for winners to perform more submission attempts, which correlates to winning;
* Additionally, there is a long tail, where looks like winners still have the edge, let's go into descriptives to be sure:

In [ ]:
examine_feature_mean_std('sub_att')

Observations: 
* Not a lot of submission attempts going on, thus min, 1st, and 2nd quartiles are all 0;
* Mean and std is slightly higher for winners;
* But look at the 3rd quartile and max, that's what we couldn't see clearly on our histogram before. The 3rd quartile is completely dominated by the winners, meaning only winners try at least 1 submission. Max also indicates that winners do more submissions than losers, which correlates to winning.

#### Reversals

In [ ]:
examine_feature_kde('rev')

Observations:
* We can see clearly that reversals are not that common. The 0 tick has an extremely high peak, which indicates a lot of 0 entries for both, winners and losers;
* We can see a bit more reversals on the 1 tick for winners;
* Around the same small peak at tick 2, to see better, we'll take a look at descriptives;
* And a long tail, where, looks like, winners are dominating, let's take a closer look:

In [ ]:
examine_feature_mean_std('rev')

Observations:
* Clearly all 3 quartiles are 0, due to fighters, mostly, not doing even a single reversal;
* Mean and std is around the same;
* But the max is higher for winners, so when fighters do a lot of reversals during a fight, winners do it more often, which can indicate a wrestling dominance and skill difference. Which in turn, *might* correlate to winning

#### Fight Control

Data types:

In [ ]:
examine_feature_kde('ctrl', log_transform=True)

Observations:
* A much higher peak at 0 tick for losers, indicating that losers more often have 0 control;
* Higher skewed distribution for winners, they peak more at the 0.5 - 2 range, then almost equal at the 2 - 5 range, and then again winners dominate in control at the 5 - 7 range.

In [ ]:
examine_feature_mean_std('ctrl')

Observations:
* All of the descriptive statistics data points are higher for winners than losers, fight control clearly correlates to winning;
* Standard deviation is higher for winners, as winner feature has less size at 0 tick and more size at extreme ticks.

# Answering questions

## Questions

1. What are the most popular locations where UFC hosts its events?
2. What is the historical average amount of UFC fights hosted annually?
3. What are the top fight outcomes?
4. Does being in a particular corner contribute to a victory? (This one is a meme, but we in that mood today:))
5. What are the main fight tactics contributing to a victory?

## Answers

1. Las Vegas, Nevada, USA takes a whopping 34.6% of all the possible UFC event locations from 1994 - 2024 (not finished yet at the time of writing this notebook);
A few other top 5 locations are (in descending order):
Abu Dhabi, Abu Dhabi, United Arab Emirates;
London, England, United Kingdom;
Anaheim, California, USA;
Atlantic City, New Jersey, USA.

    We cannot define the potential reasons for this, as additional external data is needed. But likely regulation/business reasons play a role.
    
       
2. Initially, from 1994 - 2004 there were around 5 events on average;
   Then, from 2004 till its peak in 2014 there was rapid growth with an average annual increase of 3.6 events;
   Which followed by a small drop of 5 events in 2015, after which it was in a relative plateau with fluctuations of 2-3 events on average;
   Ends with 2024, which is not finished yet, but is on track to fit in with the latest trend.

   
   We can not provide the actual reasons for this, as this requires data outside the scope of this dataset. But if I were to guess why the rapid growth starting from 2004 was due to the start of adoption of UFC as a sport, and regulatory looseness in other states.


3. Top fight outcomes are (in descending order): Decision - Unanimous, KO/TKO, Submission, Decision - split, TKO - Doctor's Stoppage, Decision - Majority.
Unanimous decisions are the top fight outcome, and the reason is, probably, that it is not that easy to KO/TKO/Submit a fighter in such a high-level competition.



4. 65% of all victories are coming from the fighters in the red corner, and the remaining 35% are coming from the blue corner fighters.
This happens due to how fighters are assigned a certain corner, red corner is given to the champions defending their title, and also red corners are given to the higher-ranked fighters. 


5. Winners in general both outstrike and outwrestle losers, which is kind of obvious. But let's get deeper:


   Let's talk striking first:
   Winners score more knockdowns, significant strikes, and total strikes. But looking even further at the significant strikes by position, we saw that winners tend to land their significant strikes to the head, whereas losers tend to keep it diverse, they land their significant strikes more to the body and legs. Also when looking at the significant strikes by position, we can see that losers actually land more from the distance, a pretty equal pct of significant strikes from the clinch position, and winners land more strikes from the ground position.

   Now let's talk wrestling:
   Winners have almost twice the losers' wrestling dominance, indicating that wrestling in general contributes to a fighting victory. But again, let's get deeper into features:
   Winners land almost twice the loser's successful takedowns, and submission attempts, and even though not as much of a difference here, but still more reversals.

   Winners also have twice as much fight control as losers.

   #### Conclusion
   
   Based on this, a fighter should focus on:
   1. All aspects of wrestling (Takedowns, submission attempts, reversals);
   2. Some aspects of striking (Knockdowns, number of total strikes, significant strikes to the head, significant strikes from the ground position);
   3. Fight control time.